In [2]:
import networkx as nx
import os

In [5]:
def read_pajek(filename, label_parser = None):

    """
    Read a Pajek format file (.net) and create a NetworkX graph.

    Parameters:
    -----------
    filename : str
        The name of the Pajek file (without extension) to be read.

    data_folder : str
        The path to the folder containing the Pajek file.

    label_parser : Callable[[str, str], Dict[str, Any]], optional
        A function used to parse node labels and values from the Pajek file.
        If not specified, a default label_parser function is used.

    Returns:
    --------
    G : nx.Graph
        A NetworkX graph representing the structure defined in the Pajek file."""
        
    filename = os.path.splitext(filename)[0]

    if label_parser is None:
        def label_parser(lab, val): return \
            {"label": lab} if val is None else {"label": lab, "value": val}

    with open(os.path.join("networks/",  f"{filename}.net"), encoding="UTF-8") as file:
        file.readline()
        nodes = []

        for line in file:
            if line.startswith("*"):
                match line.split()[0][1:]:
                    case "edges": G = nx.MultiGraph(name=filename)
                    case "arcs": G = nx.MultiDiGraph(name=filename)
                    case link_type: raise SyntaxError(f"invalid link type: {link_type}")
                break
            else: # add node
                match line.strip().split("\""):
                    case num, lab:
                        nodes.append((int(num) - 1, label_parser(lab, None)))
                    case num, lab, val:
                        nodes.append((int(num) - 1, label_parser(lab, val)))
                    case _:
                        raise SyntaxError("failed to parse " + line)

        G.add_nodes_from(nodes)

        for line in file:
            i, j = (int(x) - 1 for x in line.split()[:2])
            G.add_edge(i, j)

    return G

In [7]:
G = read_pajek("movies_graph.net")

In [10]:
zero_degree_nodes = [node for node, degree in G.in_degree() if degree == 0]

AttributeError: 'MultiGraph' object has no attribute 'in_degree'

In [9]:
nx.dfs_successors(G, source=zero_degree_nodes[0])

IndexError: list index out of range